In [10]:
import pandas as pd
from pathlib import Path

# ===== 1. 文件路径 =====
input_path  = input_path = Path("C:/Users/elvinli/OneDrive/CodeProjects/TIL6022-Group23-SAIL-Dashboard/outputs_fast/merged_sensor_all.xlsx")
 # 你的原始文件
output_path = Path("C:/Users/elvinli/OneDrive/CodeProjects/TIL6022-Group23-SAIL-Dashboard/outputs_fast/merged_sensor_all_localtime.csv") # 输出CSV路径

# ===== 2. 读取Excel =====
df = pd.read_excel(input_path, engine="openpyxl")
print("✅ 文件读取成功，形状：", df.shape)

# ===== 3. 时间转换：UTC → Europe/Amsterdam (+02:00) =====
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
df["timestamp"] = df["timestamp"].dt.tz_convert("Europe/Amsterdam")

# 👉 如果你不想显示“+02:00”，而只要当地时间的数字形式（去掉时区信息），请改为：
# df["timestamp"] = df["timestamp"].dt.tz_convert("Europe/Amsterdam").dt.tz_localize(None)

# ===== 4. 输出CSV =====
df.to_csv(output_path, index=False, float_format="%.6f")
print(f"✅ 已输出CSV：{output_path}")


✅ 文件读取成功，形状： (2400, 41)
✅ 已输出CSV：C:\Users\elvinli\OneDrive\CodeProjects\TIL6022-Group23-SAIL-Dashboard\outputs_fast\merged_sensor_all_localtime.csv


In [12]:
import pandas as pd
from pathlib import Path

# ========= 路径 =========
IN_CSV  = Path("C:/Users/elvinli/OneDrive/CodeProjects/TIL6022-Group23-SAIL-Dashboard/outputs_fast/merged_sensor_all_localtime.csv")  # 你刚导出的“当地时间”版本
OUT_CSV = Path("C:/Users/elvinli/OneDrive/CodeProjects/TIL6022-Group23-SAIL-Dashboard/outputs_fast/merged_sensor_all_localtime_with_time_feats.csv")

# ========= 读取 =========
df = pd.read_csv(IN_CSV, low_memory=False)

# 统一解析 timestamp（兼容带 +02:00 或无时区两种情况）
ts = pd.to_datetime(df["timestamp"], errors="coerce")
# 如果时间没有时区，但你确定这是“当地时间”，可取消下一行注释强制设为欧洲时区
# ts = ts.dt.tz_localize("Europe/Amsterdam", nonexistent="shift_forward", ambiguous="NaT")

df["timestamp"] = ts

# ========= 生成派生时间特征（已考虑时区）=========
dt = df["timestamp"].dt
df["year"]          = dt.year
df["month"]         = dt.month
df["day"]           = dt.day
df["hour"]          = dt.hour
df["minute"]        = dt.minute
df["weekday"]       = dt.weekday                   # Monday=0 ... Sunday=6
df["is_weekend"]    = (df["weekday"] >= 5).astype("int8")

# ISO 周序号（pandas 1.1+）
iso = dt.isocalendar()
df["week"]          = iso.week.astype("int16")

# 一天中的第几分钟（0..1439）
df["minute_of_day"] = (df["hour"] * 60 + df["minute"]).astype("int16")

# 仅日期（可选，方便 groupby）
df["date"]          = dt.date.astype("string")

# ========= 导出 =========
df.to_csv(OUT_CSV, index=False, float_format="%.6f")
print(f"✅ 已保存：{OUT_CSV}  形状={df.shape}")


✅ 已保存：C:\Users\elvinli\OneDrive\CodeProjects\TIL6022-Group23-SAIL-Dashboard\outputs_fast\merged_sensor_all_localtime_with_time_feats.csv  形状=(2400, 51)
